In [ ]:
%matplotlib widget
from pathlib import Path
import numpy as np
import pandas as pd
from tqdm import tqdm
from matplotlib import pyplot as plt
from matplotlib import gridspec
# import matplotlib.gridspec as gridspec
# from scipy.optimize import curve_fit

from lotr import LotrExperiment, DATASET_LOCATION

from lotr.plotting import despine, add_scalebar, get_circle_xy, color_stack, add_cbar, dark_col
from lotr.pca import pca_and_phase, fit_phase_neurons
from lotr.rpca_calculation import get_normalized_coords, reorient_pcs, match_rpc_and_neuron_phases
from lotr.utils import reduce_to_pi
from lotr.pca import fictive_heading_and_fit 
from lotr.utils import zscore, get_vect_angle

from itertools import product
def match_rpc_and_neuron_phases(rpc_phases, neuron_phases):
    """Function to match phase fit from neuron's best activation
    over network trajectory to neuron phase in rPC.

    Parameters
    ----------
    rpc_phases
    neuron_phases

    Returns
    -------

    """
    shifts = np.arange(-np.pi, np.pi, 0.1)
    coefs = [-1, 1]# np.arange(-2, 2, 0.2)
    params_list = list(product(coefs, shifts))
    residuals = np.zeros(len(params_list))
    for i, (coef, shift) in enumerate(params_list):
        new_phases = reduce_to_pi(neuron_phases * coef + shift)
        residuals[i] = np.sum(np.abs(new_phases - rpc_phases))

    return params_list[np.argmin(residuals)]

In [ ]:
# List all experiments
master_path = Path(DATASET_LOCATION)
file_list = sorted([f.parent for f in master_path.glob("*/*[0-9]_f*_gainmod/selected.h5")])

In [ ]:
path = master_path / "210924_f1" / "210924_f1_gainmod"
exp=LotrExperiment(path)
from bouter.utilities import predictive_tail_fill

beh_df = exp.behavior_log
theta_mat = beh_df.loc[:, [f"theta_0{i}" for i in range(9)]].values
beh_df.loc[:, [f"theta_0{i}" for i in range(9)]] = predictive_tail_fill(
    theta_mat
)

beh_df["tail_sum"] = (beh_df["theta_07"] + beh_df["theta_08"]) - (
    beh_df["theta_00"] + beh_df["theta_01"]
)

bouts_df = exp.bouts_df

traces = exp.traces[:, exp.hdn_indexes]

# Compute PCA in population dim
pcaed, phase, pca, circle_params = pca_and_phase(traces)

# Compute preferred phase of each neuron:
neuron_phases, _ = fit_phase_neurons(traces, phase, disable_bar=True)

# Compute PCA in time, fit circle and center projections:
pcaed_t, phase_t, pca_t, circle_params_t = pca_and_phase(traces.T)
cpc_scores = pcaed_t[:, :2] - circle_params_t[:2]

coords = exp.coords[exp.hdn_indexes, :]
w_coords = get_normalized_coords(coords)

# rotate pcs:
rpc_scores = reorient_pcs(cpc_scores, w_coords)

# We can now calculate a phase for each neuron from their position in this rotated space:
rpc_phases = np.angle(rpc_scores[:, 0] + 1j * rpc_scores[:, 1])

min_params = match_rpc_and_neuron_phases(rpc_phases, neuron_phases)

fict_head, params = fictive_heading_and_fit(np.unwrap(phase), bouts_df, fn=5, min_bias=0.05)

norm_activity = get_normalized_coords(traces.T).T
avg_vects = np.einsum("ij,ik->jk", norm_activity.T, rpc_scores[:, :2])

angles = get_vect_angle(avg_vects.T) #[:, 0], pcaed_t[:, 1]

In [ ]:
def load_and_plot(exp):
    # bad = [28, 26, 14, 2, 6, 19, 23, 25]

    #plt.figure()
    #plt.scatter(reduce_to_pi(min_params[0]*rpc_phases + min_params[1]), neuron_phases)
    print(np.sum(np.abs(reduce_to_pi(min_params[0]*rpc_phases + min_params[1])-neuron_phases)),
         min_params)
    
    fig = plt.figure(figsize=(8, 8))
    gs = gridspec.GridSpec(4, 2, figure=fig)
    
    beh_plot = fig.add_subplot(gs[0, :])
    beh_plot.plot(exp.behavior_log.t.values[::3], exp.behavior_log.tail_sum.values[::3])
    beh_plot.set_xlim(0, exp.behavior_log.t.values[-1])
    beh_traces = fig.add_subplot(gs[1, :], sharex=beh_plot)
    beh_traces.imshow(traces[::4, np.argsort(rpc_phases)].T, extent=(0, exp.behavior_log.t.values[-1],
                                                                  0, len(rpc_phases)), 
                      aspect="auto", cmap="gray_r")
    
    ax_plot = fig.add_subplot(gs[2, :], sharex=beh_plot)
    axs_phase = fig.add_subplot(gs[3, 0])
    axs_circle = fig.add_subplot(gs[3, 1])
    
    axs_circle.scatter(rpc_scores[:, 0], rpc_scores[:, 1], c=neuron_phases, cmap="twilight")
    axs_circle.axis("equal")
    axs_phase.scatter(pcaed[:, 0], pcaed[:, 1], c=angles, cmap="twilight", s=8)
    axs_phase.axis("equal")
    
    x = np.arange(len(phase)) / exp.fn
    ax_plot.plot(x, -zscore(np.unwrap(phase)))
    ax_plot.plot(x, zscore(fict_head))
    ax_plot.plot(x, -zscore(np.unwrap(angles)))
    title = f"{path.name}, {min_params[0]}"

    plt.suptitle(title)

In [ ]:
# path = master_path / "210314_f1" / "210314_f1_natmov"# "210926_f0" / "210926_f0_gainmod" 
# for i, path in enumerate(tqdm(file_list)):
load_and_plot(exp)

In [ ]:
bouts_df = exp.bouts_df
regr_df = exp.motor_regressors

THR_R = 0.3
THR_L = 0.5

OFF = 0.
directions = "lf", "rt"
bout_sel = dict(rt=(regr_df["right_1"] > (regr_df["left_1"] + OFF)) & (regr_df["right_1"] > THR_R),
                lf=(regr_df["left_1"] > (regr_df["right_1"] + OFF))  & (regr_df["left_1"] > THR_L))

In [ ]:
f, axs = plt.subplots(1,2, figsize=(6, 3))
s = 15
for i, coords in enumerate([[regr_df["right_1"], regr_df["left_1"]],
                           [exp.coords[:, 1], exp.coords[:, 2]]]):
    axs[i].scatter(coords[0], coords[1], c=(0.8,)*3, s=s)
    axs[i].scatter(coords[0][exp.hdn_indexes], coords[1][exp.hdn_indexes], fc=(0.8,)*3, ec=(0.3,)*3,
                  lw=0.5, s=s, label="HDNs")

    for k, sel in bout_sel.items():
        axs[i].scatter(coords[0][sel], coords[1][sel], s=s, label=k + "_sel")
    axs[i].axis("equal")


axs[0].plot([0, 0.5], [0, 0.5], "k", lw=0.5)
axs[0].legend(frameon=False, fontsize=7)
axs[0].set(xlabel="Left bouts corr.", ylabel="Right bouts corr.")
axs[1].axis("off")
plt.tight_layout()

In [ ]:
f, axs = plt.subplots(1, 1, figsize=(8, 2.5))
x = np.arange(exp.n_pts) / exp.fn

plt.plot(x, exp.traces[:, bout_sel["lf"]], c="b")
plt.plot(x, exp.traces[:, bout_sel["rt"]], c="r")
plt.plot(x, exp.traces[:, bout_sel["rt"]], c="r")
plt.plot(x, np.unwrap(phase))
plt.plot(exp.behavior_log.t.values[::3], exp.behavior_log.tail_sum.values[::3])

In [ ]:
head

In [ ]:
f, axs = plt.subplots(2, 1, figsize=(8, 6), sharex=True)

for i, k in enumerate(["rt", "lf"]):
    axs[i].plot(x, exp.traces[:, bout_sel[k]])
    axs[i].plot(x[1:], np.diff(np.unwrap(phase))*50 - 5, c=(0.3,)*3)
    # axs[i].axhline(5, lw=0.4)
    axs[i].plot(x[1:], np.diff(fict_head) - 5, lw=0.5, c="r")
    
    axs[i].plot(exp.behavior_log.t.values, exp.behavior_log.tail_sum.values + 5, c=(0.6,)*3)

In [ ]:
zscored_traces = exp.traces.copy()

In [ ]:
zscored_traces = (zscored_traces - np.nanmean(zscored_traces, 0)) / np.nanstd(zscored_traces, 0)

In [ ]:
diff_dir = zscored_traces[:, bout_sel["rt"]].mean(1) - zscored_traces[:, bout_sel["lf"]].mean(1)

In [ ]:
from lotr.utils import pearson_regressors

In [ ]:
def sigmoid(x, t=1, amp=2):
    return amp*(1 / (1 + np.exp(-t*x)) - 0.5)

regressors = np.zeros((exp.n_pts, 1))

regressors[1:, 0] = np.diff(np.unwrap(phase))

reg_coefs = pearson_regressors(zscored_traces, regressors)
reg_sorted_idxs = np.argsort(reg_coefs[0, :])
n_best = 10
neg = zscored_traces[:, reg_sorted_idxs[:n_best]].mean(1)
pos = zscored_traces[:, reg_sorted_idxs[-n_best:]].mean(1)
neg[neg < -20] = 0
pos[pos < -20] = 0
regr_based_diff = pos - neg# *0.5
# regr_based_diff[

In [ ]:
tinkered_head = np.diff(fict_head)
tinkered_head[tinkered_head < 0] = tinkered_head[tinkered_head < 0]*0.2
tinkered_head = np.cumsum(tinkered_head)

f, axs = plt.subplots(2, 1, figsize=(8, 9))

t_arr = np.arange(exp.n_pts) / exp.fn


axs[0].plot(t_arr, pos, label="best on lf.reg.")
axs[0].plot(t_arr, neg, label="best on rt.reg")
axs[0].plot(t_arr, regr_based_diff-5, label="motor ROIs. diff")
axs[0].plot(exp.behavior_log.t.values, exp.behavior_log.tail_sum.values + 10, c=(0.6,)*3, label="tail sum")
axs[0].plot(t_arr[1:], zscore(np.diff(np.unwrap(phase)))-10, label="d(phase)/dt")

axs[0].legend(frameon=False)
add_scalebar(axs[0], xunits="s", ylen=0, ylabel="", xpos=1000)

axs[1].plot(1, 1, label="_nolegend_")
axs[1].plot(1, 1, label="_nolegend_")
axs[1].plot(t_arr, zscore(np.cumsum(regr_based_diff)), label="motor ROIs. diff")
axs[1].plot(t_arr[1:], zscore(tinkered_head), c=(0.6,)*3, label="fictive head.")
axs[1].plot(t_arr, zscore(np.unwrap(phase)), label="phase")

axs[1].legend(frameon=False)
add_scalebar(axs[1], xunits="s", ylen=0, ylabel="", xpos=1000)

In [ ]:
tinkered_head = np.diff(fict_head)
tinkered_head[tinkered_head < 0] = tinkered_head[tinkered_head < 0]*0.2
tinkered_head = np.cumsum(tinkered_head)

f, axs = plt.subplots(2, 1, figsize=(8, 5))

t_arr = np.arange(exp.n_pts) / exp.fn

axs[0].plot(t_arr, zscore(fict_head), c=(0.6,)*3, label="fictive head.")
axs[0].plot(t_arr, zscore(np.unwrap(phase)), label="phase")
despine(axs[0], "all")
axs[0].set(xlim=(0, 2500), title="equal l/r bias")

axs[1].plot(t_arr[1:], zscore(tinkered_head), c=(0.6,)*3, label="fictive head.")
axs[1].plot(t_arr, zscore(np.unwrap(phase)), label="phase")
axs[1].set(xlim=(0, 2500), title="unequal l/r bias")

axs[1].legend(frameon=False)
add_scalebar(axs[1], xunits="s", ylen=0, ylabel="", xpos=2000)

In [ ]:
f, axs = plt.subplots(2, 1, figsize=(8, 4))

axs[0].plot(zscore(np.diff(np.unwrap(phase))))
axs[1].plot(zscore((phase)))
axs[1].plot(zscore(fict_head))

for t in [0.1, 0.5, 1, 2, 3]:

    predict = sigmoid(regr_based_diff, t=t)#(regr_based_diff)*np.sqrt(np.abs(regr_based_diff))
    axs[0].plot(zscore(predict))

    axs[1].plot(zscore(np.cumsum(predict)))
# plt.plot(zscored_traces[:, )
# plt.plot(zscored_traces[:, np.argsort(reg_coefs[0, :])[-10:]].mean(1))

In [ ]:
plt.figure()
plt.plot(zscore(np.cumsum(diff_dir)))
plt.plot(zscore(np.unwrap(phase)))
plt.plot(zscore(fict_head))